In [13]:
!pip install simpy

In [22]:
import simpy

In [15]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [16]:
env = simpy.Environment()

In [17]:
env.process(car(env))

<Process(car) object at 0x231471e3ba8>

In [18]:
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [25]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
    
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [26]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [63]:
import random
from statistics import mean

NEW_CUSTOMERS = 50  # Total number of customers
INTERVAL_CUSTOMERS = 8.0  # Generate new customers roughly every x minutes
MIN_PATIENCE = 5  # Min. customer patience (minutes)
MAX_PATIENCE = 8  # Max. customer patience (minutes)
MIN_SERVICE_TIME = 5 #Min. time the customer may take to be served (minutes)
MAX_SERVICE_TIME = 15 #Max. time the customer may take to be served (minutes)
OPEN_SERVICE = 300 #Time the service is open for customers (minutes)
MODULES = 1 #Amount of modules to serve customers

time_patience_lost = []
time_get_service = []

def source(env, number, interval, module):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, module, time_in_bank = random.uniform(MIN_SERVICE_TIME, MAX_SERVICE_TIME), time_patience_lost = time_patience_lost, time_get_service = time_get_service)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, module, time_in_bank, time_patience_lost, time_get_service):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with module.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            time_get_service.append(wait)
            
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))
            

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            time_patience_lost.append(wait)

# Setup and start the simulation
print('Service renege')
env = simpy.Environment()

# Start processes and run
module = simpy.Resource(env, capacity = MODULES)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, module))
env.run(until = OPEN_SERVICE)

print("\nCustomers served: " + str(len(time_get_service)))
print("Customers lost: " + str(len(time_patience_lost)))

print("\nAverage of time to be served: " + str(round(mean(time_get_service), 2)))
print("Average of patience time: " + str(round(mean(time_patience_lost), 2)))

print("\nOriginal Milestones of customers to arrive per day: " + str(NEW_CUSTOMERS))
print("Customers that arrived: " + str(len(time_get_service) + len(time_patience_lost)))

Service renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 0.0640 Customer01: Here I am
 6.4558 Customer01: RENEGED after  6.392
 7.8621 Customer02: Here I am
 8.7372 Customer00: Finished
 8.7372 Customer02: Waited  0.875
11.1109 Customer02: Finished
13.3017 Customer03: Here I am
13.3017 Customer03: Waited  0.000
13.3913 Customer03: Finished
24.7348 Customer04: Here I am
24.7348 Customer04: Waited  0.000
26.8569 Customer05: Here I am
31.8613 Customer05: RENEGED after  5.004
41.3877 Customer04: Finished
50.9828 Customer06: Here I am
50.9828 Customer06: Waited  0.000
52.2530 Customer06: Finished
66.1069 Customer07: Here I am
66.1069 Customer07: Waited  0.000
66.7430 Customer07: Finished
68.2831 Customer08: Here I am
68.2831 Customer08: Waited  0.000
78.2318 Customer09: Here I am
84.0996 Customer10: Here I am
85.3344 Customer09: RENEGED after  7.103
89.3543 Customer10: RENEGED after  5.255
106.3326 Customer08: Finished
113.4694 Customer11: Here I am
113.4694 Customer1